<a href="https://colab.research.google.com/github/ivans14/Deep_Learning_Proj/blob/main/CNN_using_VGGM_for_ball_and_player_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ball and players detection for SoccerNet
### Using CNN and the pretained model VGGM

In [1]:
import numpy as np
import tkinter
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
import seaborn as sns
from sklearn import metrics
import torch.optim as optim
import torch
from torch import nn
from torch.nn import LocalResponseNorm
import pandas as pd
import torch
import torchvision
from torchvision import transforms as torchtrans  
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import os
import cv2
from files.engine import train_one_epoch, evaluate, hi

import files.utils as utils
import files.transforms as T
from tkinter import *
# for image augmentations
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from sklearn import metrics

In [9]:
hi()

hi


False

In [7]:
# Import dataset as: train, validation and test splits

# data = pd.read_csv('proj_test/proj_det/det.txt')
# images= ['proj_test/proj_img1/{}.jpg'.format(str(i).zfill(6)) for i in range(1,750)]
# images[1]
# int(images[1][-10:-4])

# Data split
directory_train = 'proj_test/train'

# Load Train data
train_labels = pd.read_csv(directory_train + "/proj_det/det.txt", sep=',')
train_images = directory_train + "/proj_img1/{}.jpg"
train_images = [train_images.format(str(i).zfill(6)) for i in train_labels.index.values]
train_labels = [train_labels.columns.values.tolist()] + train_labels.values.tolist() # Convert DF to list

#print(train_labels[0:5][0:7])

# Load Test data
directory_test = 'proj_test/test'

test_labels = pd.read_csv(directory_test + '/proj_det/det.txt', sep=',', index_col=0)
test_images = directory_test + "/proj_img1/{}.jpg"
test_images = [test_images.format(str(i).zfill(6)) for i in test_labels.frame.values]
test_labels = [test_labels.columns.values.tolist()] + test_labels.values.tolist() # Convert DF to list

In [14]:
# Create image dataset class
class img_dataset(torch.utils.data.Dataset):
    def __init__(self, height, width, dir_img, labels_list, images, transforms=None, mode='train') -> None:
        self.transforms = transforms
        self.height = height
        self.width = width
        self.dir_img = dir_img
        self.labels_list = labels_list
        self.images=images

    def __getitem__(self,id):
        boxes = []
        labels = []
        img_name = self.images[id]
        

        # reading the images and converting them to correct size and color    
        img = cv2.imread(img_name)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img_res = cv2.resize(img_rgb, (self.width, self.height), cv2.INTER_AREA)
        # diving by 255
        img_res /= 255.0
        
        for line in range(1, len(self.labels_list)):
            #parsed = [float(x) for x in line.split(',')]
            if int(self.labels_list[line][0]) == int(self.images[id][-10:-4]):
                x = self.labels_list[line][2]
                y = self.labels_list[line][3]
                width = self.labels_list[line][4]
                height = self.labels_list[line][5]
                x_max = x + width
                y_max = y + height
                boxes.append([x,y,x_max,y_max])
                labels.append(self.labels_list[line][7])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        print(boxes)
        print(boxes.shape)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((boxes.shape[0],), dtype=torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target["iscrowd"] = iscrowd
        target["area"] = area
        image_id = torch.tensor([id])
        target["image_id"] = image_id
        target["iscrowd"] = iscrowd

        if self.transforms:
            sample = self.transforms(image = img_res,
                                        bboxes = target['boxes'],
                                        labels = labels)
            img_res = sample['image']
            target['boxes'] = torch.Tensor(sample['bboxes'])
            return img_res, target

        return img_res,boxes
         

    def __len__(self):
        return len(self.images)

In [16]:
# Create transformations fo the dataset to increase the data.
# Send train=True for training transforms and False for val/test transforms
def get_transform(train):
  if train:
    return A.Compose(
      [
        A.HorizontalFlip(0.5),
        # ToTensorV2 converts image to pytorch tensor without div by 255
        ToTensorV2(p=1.0) 
      ],
      bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']}
    )
  else:
    return A.Compose(
      [ToTensorV2(p=1.0)],
      bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']}
    )

In [17]:
train_ds = img_dataset(1080,1920,directory_train + "/proj_img1/",train_labels,train_images, transforms=get_transform(train=True))
test_ds = img_dataset(1080,1920,directory_test + "/proj_img1/",test_labels,test_images, mode='test', transforms=get_transform(train=False))
ran_img, boxes = train_ds[20]
ran_img.shape

tensor([[ 929.,  488.,  974.,  624.],
        [ 401.,  344.,  434.,  450.],
        [   4.,  328.,   42.,  431.],
        [ 253.,  537.,  308.,  695.],
        [ 757.,  529.,  809.,  671.],
        [ 302.,  865.,  406., 1079.],
        [  24.,  666.,  154.,  835.],
        [ 114.,  333.,  148.,  426.],
        [ 735.,  244.,  770.,  321.],
        [1537.,  525., 1577.,  663.],
        [1255.,  610., 1277.,  634.],
        [1229.,  607., 1281.,  791.]])
torch.Size([12, 4])


torch.Size([3, 1080, 1920])

In [18]:
#Visualize the data
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def plot_img_bbox(img, boxes):
  # plot the image and bboxes
  # Bounding boxes are defined as follows: x-min y-min width height
  fig, a = plt.subplots(1,1)
  fig.set_size_inches(5,5)
  # img_plot = mpimg.imread(img)
  a.imshow(img)
  count = 0
  for box in range(0,boxes.shape[0]):
    print(box)
    print(box[1], " and ", box[2])
    x, y, width, height  = float(box[0]), float(box[1]), float(box[2]) - float(box[0]), float(box[3]) - float(box[1])
    rect = patches.Rectangle(
      (x, y),
      width, height,
      linewidth = 2,
      edgecolor = 'r',
      facecolor = 'none'
    )
    # Draw the bounding box on top of the image
    a.add_patch(rect)
    a.annotate(str(count), xy=(x,y))
    count += 1
  plt.show()

# print(boxes)
# img = mpimg.imread(ran_img)
# imgplot = plt.imshow(img)
# plt.show()

plot_img_bbox(ran_img, boxes)

TypeError: Invalid shape (3, 1080, 1920) for image data

In [7]:
# # CNN model
# out_features = 2 # Ball and players
# image_shape = (400, 400, 3)

# class Model(nn.Module):
#     def __init__(self,out_classes, image_size):
#         super().__init__()
#         self.out_classes = out_classes
#         H = image_size[0]
#         W = image_size[1]
#         C = image_size[2]

#         conv1 = {
#             "in": 1,
#             "out": 16,
#             "kernel": 5,
#             "stride": 2,
#             "padding": 0
#         }
#         conv2 = {
#             "in":conv1["out"],
#             "out": 28,
#             "kernel": 5,
#             "stride": 2,
#             "padding": 0
#         }
#         pool = {
#             "kernel": 2,
#             "stride": 2,
#             "padding": 0
#         }

#         conv_param = [conv1,pool,conv2,pool]

#         for i in range (0, len(conv_param)):
#             H = np.floor((H + 2*conv_param[i]["padding"] - conv_param[i]["kernel"])/conv_param[i]["stride"] + 1)
#             W = np.floor((W + 2*conv_param[i]["padding"] - conv_param[i]["kernel"])/conv_param[i]["stride"] + 1)


#         activation_fn = nn.ReLU(0.1)
#         # Defining a Sequential pipeline for the entire CNN
#         self.net = nn.Sequential(
#             nn.Conv2d(in_channels = conv_param[0]["in"],
#                      out_channels = conv_param[0]["out"],
#                      kernel_size = conv_param[0]["kernel"],
#                      stride= conv_param[0]["stride"],
#                      padding= conv_param[0]["padding"]),
#             activation_fn,
#             LocalResponseNorm(2),   # Check: https://towardsdatascience.com/difference-between-local-response-normalization-and-batch-normalization-272308c034ac
#             nn.MaxPool2d(kernel_size = conv_param[1]["kernel"],
#                         stride =conv_param[1]["stride"]),
            
#             nn.Conv2d(in_channels = conv_param[2]["in"],
#                      out_channels = conv_param[2]["out"],
#                      kernel_size = conv_param[2]["kernel"],
#                      stride= conv_param[2]["stride"],
#                      padding= conv_param[2]["padding"]),
#             nn.MaxPool2d(kernel_size = conv_param[3]["kernel"],
#                         stride =conv_param[3]["stride"]),
            
#             #nn.Flatten(), # Flattens a contiguous range of dims into a tensor.
#             # FFNN
#             activation_fn,
#             nn.Linear(int(conv_param[-2]["out"] * H * W), out_features),
#             # nn.Dropout(0.5),
#             # nn.Sigmoid(),
#             # nn.Linear(,self.out_classes),
#             # nn.Dropout(0.25),
#             # nn.Sigmoid()
#         )


#     def forward(self, x):

#       return nn.Softmax(self.net(x))

# NN = Model(out_features, image_shape)
# device = device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')  # use cuda if possible
# NN.to(device)
# print(NN)

# # Definition of optimizer and loss functions
# optimizer = optim.Adam(NN.parameters(),lr=0.008)
# loss_fn = nn.CrossEntropyLoss() 

## DATALOADER

In [19]:
# dataset = img_dataset(1080,1920,'proj_test/proj_img1/',transforms=get_transform(train=True))
# dataset_test = img_dataset(1080,1920,'proj_test/proj_img1/',transforms=get_transform(train=True))

# # split the dataset in train and test set
# torch.manual_seed(1)
# indices = torch.randperm(len(dataset)).tolist()

# # train test split
# test_split = 0.2
# tsize = int(len(dataset)*test_split)
# dataset = torch.utils.data.Subset(dataset, indices[:10])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-tsize:])

# define training and validation data loaders
data_loader = torch.utils.data.DataLoader(
  train_ds,
  batch_size=1,
  shuffle=True,
  num_workers=0,
  collate_fn=utils.collate_fn,
)

data_loader_test = torch.utils.data.DataLoader(
  test_ds,
  batch_size=10,
  shuffle=False,
  num_workers=0,
  collate_fn=utils.collate_fn,
)

PRETRAINED MODEL


In [40]:
## Faster RESNET 50 pre-trained model :)
def get_object_detection_model(num_classes):
  # load a model pre-trained pre-trained on COCO
  model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
  # get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # replace the pre-trained head with a new one
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) 
  return model

TRAINING


In [41]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

num_classes = 2 #CLASS ZERO FOR BACKGROUND, 1 FOR PLAYERS AND 2 FOR BALL 

# get the model using our helper function
model = get_object_detection_model(num_classes)

# move model to the right device
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(
  optimizer,
  step_size=3,
  gamma=0.1
)

In [12]:
torch.cuda.is_available()

False

In [42]:
# training for 5 epochs
num_epochs = 1

for epoch in range(num_epochs):
    # training for one epoch
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    # evaluate on the test dataset
    evaluate(model, data_loader_test, device=device)

# Save model (i.e. its weights)
models_dir = "trained_models/"
torch.save(model.state_dict(), models_dir + 'CNN_weights_{}.pth'.format(len([entry for entry in os.listdir(models_dir) if os.path.isfile(os.path.join(models_dir,entry))])))
print("Finished training.")

proj_test/proj_img1/000313.jpg
Epoch: [0]  [ 0/10]  eta: 0:03:30  lr: 0.000560  loss: 0.6870 (0.6870)  loss_classifier: 0.5921 (0.5921)  loss_box_reg: 0.0000 (0.0000)  loss_objectness: 0.0730 (0.0730)  loss_rpn_box_reg: 0.0218 (0.0218)  time: 21.0822  data: 0.0996  max mem: 0
proj_test/proj_img1/000162.jpg
proj_test/proj_img1/000278.jpg
proj_test/proj_img1/000391.jpg
proj_test/proj_img1/000090.jpg
proj_test/proj_img1/000181.jpg
proj_test/proj_img1/000125.jpg
proj_test/proj_img1/000562.jpg
proj_test/proj_img1/000156.jpg
proj_test/proj_img1/000587.jpg


In [ ]:
# Define training function
# HOLA 
# adios

def accuracy(target, pred):
    return metrics.accuracy_score(target.detach().cpu().numpy(), pred.detach().cpu().numpy())

# Set number of epochs according to computational power, time and results.
batch_size = 64
num_epochs = 4
validation_every_steps = 500

step = 0
NN.train()

train_accuracies = []
valid_accuracies = []
running_loss = 0



for epoch in range(num_epochs):
    
    train_accuracies_batches = []
    train_loss_batches = []
    
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Forward pass, compute gradients, perform one training step.
        # Your code here!
        output = NN(inputs) # Pass the inouts through the NN
        loss = loss_fn(output,targets) # Compute and Save loss
        optimizer.zero_grad() #Clean up gradients
        loss.backward() # Compute gradients based on the loss from the current batch
        optimizer.step()
        running_loss += loss.item()

        # Increment step counter
        step += 1
        
        # Compute accuracy.
        predictions = output.max(1)[1]
        train_accuracies_batches.append(accuracy(targets, predictions))
        
        if step % validation_every_steps == 0:
            
            validation_loss = 0
            # Append average training accuracy to list.
            train_accuracies.append(np.mean(train_accuracies_batches))
            
            train_accuracies_batches = []
        
            # Compute accuracies on validation set.
            valid_accuracies_batches = []
            with torch.no_grad():
                NN.eval()
                for inputs, targets in test_loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    output = NN(inputs)
                    loss = loss_fn(output, targets)

                    predictions = output.max(1)[1]

                    # Multiply by len(x) because the final batch of DataLoader may be smaller (drop_last=False).
                    valid_accuracies_batches.append(accuracy(targets, predictions) * len(inputs))

                NN.train()
                
            # Append average validation accuracy to list.
            valid_accuracies.append(np.sum(valid_accuracies_batches) / len(test_set))
     
            print(f"Step {step:<5}   training accuracy: {train_accuracies[-1]}")
            print(f"             test accuracy: {valid_accuracies[-1]}")

# Save model (i.e. its weights)
models_dir = "trained_models/"
torch.save(model.state_dict(), models_dir + 'CNN_weights.pth')
print("Finished training.")